# Week 11 Lab: Decision Trees, Random Forests, and Feature Importance

<a href="https://colab.research.google.com/github/bradleyboehmke/uc-bana-4080/blob/main/labs/11_wk11_lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Welcome to Week 11! This lab introduces you to tree-based machine learning methods—decision trees and random forests—along with techniques for understanding which features drive your model's predictions. You'll learn how these powerful algorithms automatically discover non-linear patterns and interactions in data, making them invaluable tools for real-world business applications.

Today's lab has two distinct parts: First, your TA will walk through a complete analysis using the Boston housing dataset, demonstrating how to build, evaluate, and interpret tree-based models. Then, you'll apply these techniques independently to the Breast Cancer Wisconsin dataset, building on your classification work from Week 10.

## 🎯 Learning Objectives
By the end of this lab, you will be able to:
- Build and interpret decision tree models for regression and classification problems
- Construct random forest models and tune key hyperparameters
- Compare model performance to select the best approach for a given problem
- Calculate and interpret feature importance using permutation importance
- Create and interpret partial dependence plots to understand feature effects

## 📚 This Lab Reinforces
- **Chapter 25: Decision Trees - Foundations and Interpretability**
- **Chapter 26: Random Forests - Ensemble Power and Robustness**
- **Chapter 27: Understanding Feature Importance**

## 🕐 Estimated Time & Structure
**Total Time:** 75 minutes  
**Mode:** TA-led demonstration + group work (serves as homework)

- **[0–30 min]** Part 1: TA walkthrough with Boston housing data
- **[30–35 min]** Class Q&A and transition
- **[35–70 min]** Part 2: Independent group challenges with Breast Cancer data
- **[70–75 min]** Wrap-up and reflection

You are encouraged to work in small groups of **2–4 students** for Part 2.

## 💡 Why This Matters
Tree-based methods are among the most widely used machine learning algorithms in business. Unlike linear models that assume straight-line relationships, decision trees and random forests automatically discover complex patterns, threshold effects, and feature interactions. Banks use them for credit risk assessment, hospitals for readmission prediction, and retailers for customer churn modeling. Feature importance techniques help you understand *why* your model makes certain predictions—critical for building stakeholder trust, debugging models, and making data-driven business decisions.

## Setup
We'll work with two datasets today: the Boston housing dataset (for regression with the TA) and the Breast Cancer Wisconsin dataset (for your independent classification analysis).

In [ ]:
# Required imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.inspection import permutation_importance, PartialDependenceDisplay
from ISLP import load_data
import warnings
warnings.filterwarnings('ignore')

# Set random state for reproducibility
RANDOM_STATE = 42

print("✅ All libraries imported successfully!")
print("🌳 Ready to explore tree-based methods!")

## Part 1 — TA Walkthrough: Boston Housing Data (30 minutes)

Your TA will demonstrate the complete workflow for tree-based modeling using the Boston housing dataset. This is a regression problem where we predict median home values based on neighborhood characteristics.

**Key Steps in This Walkthrough:**
1. Load and explore the Boston housing data
2. Prepare data and create train/test split
3. Build and evaluate a decision tree regression model
4. Build and evaluate a random forest regression model
5. Calculate and interpret feature importance
6. Create partial dependence plots to understand feature effects

**Follow along and take notes** — you'll apply this same process to the Breast Cancer data in Part 2!

### Step 1: Load and Explore Data

**Context:** The Boston housing dataset contains information about housing in the Boston area. Each observation represents a neighborhood, with features like crime rate, property tax rate, and pupil-teacher ratio. Our goal is to predict the median home value (`medv`).

**Reference:** See Chapter 25, "Building Your First Decision Tree" section for data preparation details.

In [ ]:
# Load Boston housing data
Boston = load_data('Boston')

print("Boston Housing Dataset Overview:")
print(f"Shape: {Boston.shape}")
print(f"\nFeatures: {Boston.columns.tolist()}")
print(f"\nFirst few rows:")
print(Boston.head())

# Check for missing values
print(f"\nMissing values: {Boston.isnull().sum().sum()}")

# Basic statistics on target variable
print(f"\nTarget variable (medv) statistics:")
print(Boston['medv'].describe())

### Step 2: Prepare Data and Split

**Key Concept:** We separate features (X) from the target variable (y), then split into training and test sets to evaluate model performance on unseen data.

**Reference:** Chapter 25 reviews train/test splitting for tree-based models.

In [ ]:
# Prepare features and target
X_boston = Boston.drop('medv', axis=1)
y_boston = Boston['medv']

# Create 70-30 train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X_boston, y_boston, test_size=0.3, random_state=RANDOM_STATE
)

print(f"Training set: {len(X_train)} observations")
print(f"Test set: {len(X_test)} observations")
print(f"\nNumber of features: {X_train.shape[1]}")

### Step 3: Build Decision Tree Model

**Key Concept:** Decision trees make predictions by learning a series of yes/no questions about features. Unlike linear models that assume straight-line relationships, trees automatically discover non-linear patterns and interactions.

**Reference:** Chapter 25, "Regression Trees" section explains how trees minimize MSE to find optimal splits.

In [ ]:
# Build decision tree with max_depth=5 to prevent overfitting
dt_model = DecisionTreeRegressor(max_depth=5, random_state=RANDOM_STATE)
dt_model.fit(X_train, y_train)

# Make predictions
y_pred_train_dt = dt_model.predict(X_train)
y_pred_test_dt = dt_model.predict(X_test)

# Evaluate performance
train_rmse_dt = np.sqrt(mean_squared_error(y_train, y_pred_train_dt))
test_rmse_dt = np.sqrt(mean_squared_error(y_test, y_pred_test_dt))
train_r2_dt = r2_score(y_train, y_pred_train_dt)
test_r2_dt = r2_score(y_test, y_pred_test_dt)

print("Decision Tree Performance:")
print(f"Training RMSE: ${train_rmse_dt:,.2f}")
print(f"Test RMSE: ${test_rmse_dt:,.2f}")
print(f"Training R²: {train_r2_dt:.3f}")
print(f"Test R²: {test_r2_dt:.3f}")

### Step 4: Build Random Forest Model

**Key Concept:** Random forests combine many decision trees (each trained on a different random sample of data) to make more accurate and stable predictions. While a single tree can be unstable, the wisdom of crowds from many diverse trees produces robust results.

**Reference:** Chapter 26, "How Random Forests Work" section explains bootstrap sampling and feature randomness.

In [ ]:
# Build random forest with 100 trees
rf_model = RandomForestRegressor(
    n_estimators=100,
    max_depth=10,
    random_state=RANDOM_STATE
)
rf_model.fit(X_train, y_train)

# Make predictions
y_pred_train_rf = rf_model.predict(X_train)
y_pred_test_rf = rf_model.predict(X_test)

# Evaluate performance
train_rmse_rf = np.sqrt(mean_squared_error(y_train, y_pred_train_rf))
test_rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_test_rf))
train_r2_rf = r2_score(y_train, y_pred_train_rf)
test_r2_rf = r2_score(y_test, y_pred_test_rf)

print("Random Forest Performance:")
print(f"Training RMSE: ${train_rmse_rf:,.2f}")
print(f"Test RMSE: ${test_rmse_rf:,.2f}")
print(f"Training R²: {train_r2_rf:.3f}")
print(f"Test R²: {test_r2_rf:.3f}")

# Compare models
print(f"\n📊 Model Comparison:")
print(f"Random Forest improves test R² by {test_r2_rf - test_r2_dt:.3f} over single tree")

### Step 5: Calculate and Interpret Feature Importance

**Key Concept:** Feature importance tells us which variables have the strongest influence on predictions. For random forests, we'll use **permutation importance**, which measures how much performance drops when we shuffle each feature's values.

**Why This Matters:** Understanding which features drive predictions helps us build stakeholder trust, validate that the model makes business sense, and identify which factors to focus on.

**Reference:** Chapter 27, "Permutation Importance" section explains this model-agnostic approach.

In [ ]:
# Calculate permutation importance on test set
perm_importance = permutation_importance(
    rf_model,
    X_test,
    y_test,
    n_repeats=10,
    random_state=RANDOM_STATE,
    scoring='r2'
)

# Create DataFrame for easy viewing
importance_df = pd.DataFrame({
    'feature': X_train.columns,
    'importance': perm_importance.importances_mean
}).sort_values('importance', ascending=False)

print("Feature Importance (Permutation Method):")
print(importance_df)

# Visualize top 10 features
plt.figure(figsize=(10, 6))
top_features = importance_df.head(10)
plt.barh(range(len(top_features)), top_features['importance'])
plt.yticks(range(len(top_features)), top_features['feature'])
plt.xlabel('Importance (drop in R² when shuffled)')
plt.title('Top 10 Most Important Features for House Price Prediction')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

print(f"\n💡 Interpretation: The top feature is '{importance_df.iloc[0]['feature']}' with importance {importance_df.iloc[0]['importance']:.3f}")
print(f"This means shuffling this feature causes R² to drop by {importance_df.iloc[0]['importance']:.3f}")

### Step 6: Create Partial Dependence Plots

**Key Concept:** Partial dependence plots (PDPs) show how predictions change as we vary one feature while holding others constant. They reveal the *shape* of the relationship between a feature and the target.

**Why This Matters:** While feature importance tells us *which* features matter, PDPs tell us *how* they influence predictions. Do prices increase linearly with square footage? Is there a threshold effect?

**Reference:** Chapter 27, "Exploring Prediction Behavior Across Variables" section covers PDPs in detail.

In [ ]:
# Create PDP for the most important feature
top_feature = importance_df.iloc[0]['feature']

print(f"Creating Partial Dependence Plot for: {top_feature}")

# Create PDP
fig, ax = plt.subplots(figsize=(8, 5))
PartialDependenceDisplay.from_estimator(
    rf_model,
    X_train,
    features=[top_feature],
    ax=ax
)
plt.tight_layout()
plt.show()

print(f"\n💡 How to read this plot:")
print(f"- X-axis: Values of {top_feature}")
print(f"- Y-axis: Predicted median home value (in $1000s)")
print(f"- The line shows how predictions change as {top_feature} increases")
print(f"- Steep slopes indicate strong effects; flat regions indicate weak effects")

### ✅ Part 1 Summary

**What we demonstrated:**
1. ✓ Built a decision tree regression model
2. ✓ Built a random forest that outperformed the single tree
3. ✓ Identified the most important features using permutation importance
4. ✓ Visualized how the top feature influences predictions using PDPs

**Key Takeaways:**
- Random forests typically outperform single decision trees
- Feature importance helps identify which variables drive predictions
- PDPs reveal *how* features influence the outcome

**Next:** You'll apply this same workflow to the Breast Cancer dataset!

## Class Discussion/Q&A (5 minutes)

Before moving to Part 2, let's address any questions about the workflow:

**Discussion prompts:**
- How do decision trees differ from linear regression in terms of the relationships they can capture?
- Why did the random forest outperform the single decision tree?
- What does feature importance tell us that model performance metrics don't?
- How would you explain a partial dependence plot to a non-technical stakeholder?

**Common questions:**
- *"How do I choose max_depth for a decision tree?"* → Start with smaller values (3-10) and increase if needed. Too deep = overfitting.
- *"How many trees should I use in a random forest?"* → Typically 100-500. More trees = better performance but diminishing returns.
- *"What's the difference between impurity-based and permutation importance?"* → Impurity is faster but biased toward high-cardinality features; permutation is more reliable but slower.

## Part 2 — Independent Group Challenges: Breast Cancer Classification (35 minutes)

Now it's your turn! You'll apply the same tree-based modeling workflow to the **Breast Cancer Wisconsin dataset** that you worked with in Week 10. This time, instead of logistic regression, you'll use decision trees and random forests for classification.

**Scenario:** You're a data scientist at a medical diagnostics company. Your team needs to build a model that predicts whether a breast tumor is malignant (cancerous) or benign (non-cancerous) based on cell nucleus measurements from medical images. Understanding which features drive these predictions is critical for clinical validation and physician trust.

### 🚨 Important Guidelines for Part 2

For the challenges below:
- **NO starter code provided** — you'll write everything from scratch
- **DO NOT use AI tools** (ChatGPT, Copilot, etc.) to generate code
- **Work in groups of 2-4 students** and collaborate
- **Ask the instructor for help** if you get stuck
- **Refer to the chapters** for syntax and examples
- **This section serves as your homework** — complete all challenges

### 📊 Data Preparation (PROVIDED)

We'll load and prepare the Breast Cancer data for you. Your challenges begin with splitting the data.

In [ ]:
# Load Breast Cancer Wisconsin dataset (PROVIDED)
url = "https://raw.githubusercontent.com/bradleyboehmke/uc-bana-4080/refs/heads/main/data/breast_cancer.csv"
cancer_data = pd.read_csv(url)

print("✅ Breast Cancer Wisconsin dataset loaded")
print(f"Shape: {cancer_data.shape}")
print(f"\nTarget variable distribution:")
print(cancer_data['diagnosis'].value_counts())
print(f"\nBaseline: {(cancer_data['diagnosis']=='M').mean():.1%} malignant")

In [ ]:
# Prepare features and target (PROVIDED)
# Create binary target: 1=Malignant, 0=Benign
y_cancer = (cancer_data['diagnosis'] == 'M').astype(int)

# Select all numeric features (exclude diagnosis column)
X_cancer = cancer_data.drop('diagnosis', axis=1)

print(f"✅ Data prepared for modeling")
print(f"Features: {X_cancer.shape[1]} columns")
print(f"Target: {len(y_cancer)} observations")
print(f"\nFeature names:")
print(X_cancer.columns.tolist())

### Challenge 1 — Split the Data into Training and Test Sets

**Question:** Create a 70-30 train/test split of the breast cancer data. Use `RANDOM_STATE` for reproducibility and `stratify=y_cancer` to maintain the class balance in both sets.

**Your Task:**
- Split `X_cancer` and `y_cancer` into training and test sets
- Use 70% for training, 30% for testing
- Name your variables: `X_train_cancer`, `X_test_cancer`, `y_train_cancer`, `y_test_cancer`
- Print the sizes of both sets and the malignant rate in each
- **IMPORTANT**: make sure you use `random_state=RANDOM_STATE`

**Hint:** Review Chapter 25, "Regression Trees" section for the train/test split syntax. Don't forget to use `stratify=y_cancer` to ensure both sets have similar proportions of malignant/benign cases!

In [ ]:
# Set random state for reproducibility
RANDOM_STATE = 42

# Your turn: split the data


### Challenge 2 — Build Decision Trees with Different Depths

**Question:** Build decision tree classifiers with different `max_depth` values and compare their performance. This will help you understand the bias-variance tradeoff.

**Your Task:**
1. Test the following `max_depth` values: **3, 5, 10**
2. For each depth:
   - Create a `DecisionTreeClassifier` with that `max_depth` and `random_state=RANDOM_STATE`
   - Fit it on the training data
   - Calculate **training accuracy** and **test accuracy**
3. Store the results and identify which depth gives the best test accuracy
4. Create a plot showing how training and test accuracy change with depth

**Hint:** Chapter 25, "Tree Parameters and Overfitting" section explains `max_depth`. You'll need to use `DecisionTreeClassifier` (not Regressor since this is classification). Use `.score()` method to get accuracy.

**Expected pattern:** Training accuracy should increase with depth, but test accuracy may plateau or decrease (overfitting).

In [ ]:
# Your turn: test different tree depths


### Challenge 3 — Build Random Forests with Different Hyperparameters

**Question:** Now build random forest classifiers and tune hyperparameters to find the best combination for predicting breast cancer.

**Your Task:**
Test the following hyperparameter combinations:

| Model | n_estimators | max_depth | max_features |
|-------|--------------|-----------|-------------|
| RF1   | 100          | 10        | 'sqrt'      |
| RF2   | 200          | 15        | 'sqrt'      |
| RF3   | 300          | 20        | 10          |
| RF4   | 500          | None      | 'sqrt'      |

For each model:
1. Create a `RandomForestClassifier` with the specified hyperparameters (use `random_state=RANDOM_STATE`)
2. Fit on training data
3. Calculate training accuracy, test accuracy, precision, recall, and F1-score on test set
4. Store results for comparison

**Hint:** Chapter 26, "Key Hyperparameters" section explains each parameter:
- `n_estimators`: Number of trees in the forest
- `max_depth`: Maximum depth of each tree (None = unlimited)
- `max_features`: Number of features to consider at each split

**Medical Context:** For cancer diagnosis, recall (catching all malignant cases) is often more important than precision (avoiding false alarms).

In [ ]:
# Your turn: test different random forest configurations


### Challenge 4 — Identify the Best Model

**Question:** Compare all the models you've built (decision trees from Challenge 2 and random forests from Challenge 3) and identify which one performs best.

**Your Task:**
1. Create a summary table or visualization comparing:
   - All decision tree depths tested (Challenge 2)
   - All random forest configurations (Challenge 3)
2. Compare them on test accuracy, precision, recall, and F1-score
3. Identify the single best model overall
4. Explain WHY this model is best (consider: which metrics matter most for cancer diagnosis?)

**Discussion Questions:**
- Did random forests outperform single decision trees? By how much?
- Which random forest configuration worked best?
- Given that missing a malignant tumor is more serious than a false alarm, which metric should we prioritize?
- Would you recommend this model for clinical use? What would you want to improve?

**Hint:** Chapter 26 discusses why random forests typically outperform single trees (ensemble averaging reduces overfitting).

In [ ]:
# Your turn: compare all models and identify the best


### Challenge 5 — Identify Top 5 Most Important Features

**Question:** Using your best-performing random forest model from Challenge 4, identify which features are most important for predicting breast cancer.

**Your Task:**
1. Use **permutation importance** (not impurity-based) on your best random forest model
2. Calculate importance on the **test set** with `n_repeats=10`
3. Identify the **top 5 most important features**
4. Create a horizontal bar plot visualizing these top 5 features
5. Interpret the results: Do these features make medical/biological sense?

**Medical Context:** Understanding which cell characteristics drive cancer predictions helps clinicians validate the model and focus their diagnostic attention on the most informative measurements.

**Hint:** Chapter 27, "Permutation Importance" section shows the complete workflow:
```python
from sklearn.inspection import permutation_importance
perm_importance = permutation_importance(
    model, X_test, y_test, n_repeats=10, random_state=RANDOM_STATE
)
```

**Expected features:** You should see features related to cell size, shape, and texture among the top predictors.

In [ ]:
# Your turn: calculate permutation importance and identify top 5 features


### Challenge 6 — Create and Interpret Partial Dependence Plot

**Question:** Create a partial dependence plot (PDP) for the most important feature from Challenge 5 to understand HOW it influences cancer predictions.

**Your Task:**
1. Identify the #1 most important feature from Challenge 5
2. Create a partial dependence plot for this feature using your best random forest
3. Interpret the plot by answering:
   - As this feature increases, does malignancy probability increase or decrease?
   - Is the relationship linear or non-linear?
   - Are there any threshold effects (sudden changes)?
   - What does this tell us about cancer diagnosis?

**Clinical Interpretation:** If the most important feature is something like `radius_mean` (cell size), you might find that larger cells are associated with higher cancer probability. This aligns with medical knowledge that cancerous cells often grow larger than normal cells.

**Hint:** Chapter 27, "Exploring Prediction Behavior Across Variables" section explains PDPs:
```python
from sklearn.inspection import PartialDependenceDisplay
PartialDependenceDisplay.from_estimator(
    model, X_train, features=['feature_name']
)
```

**Remember:** PDPs show the average predicted probability across all observations as we vary one feature.

In [ ]:
# Your turn: create PDP for the most important feature and interpret


## 🎓 Lab Wrap-Up & Reflection

### ✅ What You Accomplished

Congratulations! In this lab, you:
- **Built decision tree models** and explored how max_depth affects the bias-variance tradeoff
- **Constructed random forest models** and tuned multiple hyperparameters (n_estimators, max_depth, max_features)
- **Compared model performance** across different configurations to select the best approach
- **Calculated permutation importance** to identify which features drive cancer predictions
- **Created partial dependence plots** to understand how features influence malignancy probability
- **Applied tree-based methods** to a real medical classification problem

### 🤔 Reflection Questions

Take 2-3 minutes to consider:

1. **Model Comparison**: Why did random forests outperform single decision trees? What advantage comes from combining many trees?

2. **Feature Importance**: Which cell characteristics were most predictive of cancer? Do these align with medical knowledge about cancer cells?

3. **Interpretability vs. Accuracy**: Decision trees are easier to visualize than random forests. Is the accuracy gain from random forests worth the loss of interpretability for medical diagnosis?

4. **Business Impact**: How would you explain your model's predictions to a physician who doesn't have data science training? What role do PDPs play in building trust?

5. **Actionable vs. Predictive**: Some features are important for prediction but not actionable (e.g., cell radius_mean). How does this distinction matter for clinical decision-making?

### 🔗 Connection to Course Goals

This lab bridges several key data science concepts:
- **Supervised learning**: Classification with tree-based methods
- **Model evaluation**: Comparing models using multiple metrics
- **Interpretability**: Understanding what drives predictions
- **Business application**: Medical diagnosis where false negatives have serious consequences

Tree-based methods are among the most widely used algorithms in practice because they:
- Handle non-linear relationships automatically
- Require minimal data preprocessing
- Provide feature importance insights
- Work well out-of-the-box with minimal tuning

### 📋 Next Steps

- **Homework**: This lab serves as your Week 11 homework. Make sure all challenges are complete and well-documented.
- **Next Week**: We'll explore gradient boosting methods (XGBoost, LightGBM) which often outperform random forests by building trees sequentially rather than independently.
- **Additional Practice**: Try applying these techniques to other datasets from previous weeks (Default, Ames housing, etc.)

---
**💾 Save your work** and ensure all code cells run successfully from top to bottom!

## 🚨 Troubleshooting & Common Issues

**Issue 1: "ImportError: cannot import name 'permutation_importance'"**
- **Solution:** Make sure you're using scikit-learn version 0.22 or higher. Update with: `pip install --upgrade scikit-learn`

**Issue 2: "My decision tree has 100% training accuracy but poor test accuracy"**
- **Solution:** This is overfitting! Your tree is too deep. Try:
  - Reducing `max_depth` to 5-10
  - Increasing `min_samples_split` to 20-50
  - Increasing `min_samples_leaf` to 10-20

**Issue 3: "Random forest takes a long time to train"**
- **Solution:** This is normal for large forests. Speed it up by:
  - Reducing `n_estimators` (100 is usually sufficient)
  - Adding `n_jobs=-1` to use all CPU cores
  - Limiting `max_depth` to prevent very deep trees

**Issue 4: "Permutation importance shows negative values"**
- **Solution:** Small negative values are normal—they mean shuffling that feature slightly improved performance by chance. These features are not important.

**Issue 5: "My PDP looks very jagged/noisy"**
- **Solution:** This happens in sparse data regions. Try:
  - Increasing `grid_resolution` parameter
  - Checking the data distribution (rug plot) to see where you have few observations
  - Focus interpretation on regions with dense data

**Issue 6: "AttributeError: 'DecisionTreeClassifier' has no attribute 'feature_importances_'"**
- **Solution:** You must `.fit()` the model before accessing feature importance. Make sure you've trained the model first.

**Issue 7: "All my random forests perform similarly regardless of hyperparameters"**
- **Solution:** This could mean:
  - The default parameters are already good for this dataset
  - The dataset is relatively simple and most reasonable configurations work well
  - Try more extreme parameter values to see bigger differences

### General Debugging Tips:

1. **Check shapes**: Always verify `X_train.shape`, `y_train.shape` match expectations
2. **Print intermediate results**: Don't wait until the end to check if things work
3. **Start simple**: Build one model successfully before trying multiple configurations
4. **Read error messages carefully**: They often tell you exactly what's wrong
5. **Use RANDOM_STATE consistently**: This ensures reproducible results for debugging
6. **Refer to the chapters**: The code examples in Chapters 25-27 show complete working syntax

### Getting Help:

If you're stuck:
1. Re-read the relevant chapter section mentioned in the challenge hint
2. Check the Part 1 TA walkthrough for similar code patterns
3. Ask your group members or neighboring groups
4. Raise your hand and ask the instructor
5. Post on Canvas discussion board with your specific error message